##Named Entity Recognition and Linking

***Introduction***

In this notebooks, we are going to show how we can analyze a text to find the mentions of named entities in it and link those named entities to their corresponding wikipedia page. This notebook is intended as a starting point, showing examples of using two pretrained models, with the goal that we can then use other models for different analysis of our text.

We will start by importing and installing required libraries. We will be using Transformers library and pre-trained models maintained by Huggingface.

We will install Transformers

In [1]:
!pip install transformers seqeval[gpu]

Now we will import various libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import helper_functions as hf
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM
from transformers import pipeline

Now we will import the models. For the task of Named Entity Recognition we will use base model of BERT NER "bert-base-NER" and for Linking we will use Facebook's mgenre-wiki.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
modelNER = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
tokenizerNEL = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")
modelNEL = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenCl

We now extract the named entities. We do that by first defining a default sentence.

In [5]:
record_id = "C17023786"
inp_id = input("Please enter the id of the record you want to analyse. Leave it blank if you want to use the default text")
if inp_id != "":
  record_id = inp_id
sentence = hf.populate_texts(record_id)
print(sentence)

Please enter the id of the record you want to analyse. Leave it blank if you want to use the default text
  Konni ZILLIACUS: British. A Russian Finn by birth, he came to notice through his Communist associations while working for the League of Nations in 1932. He was elected Labour MP for Gateshead in the 1945 General Election, but was expelled from the party in 1949 for persistently attacking party policy 


In the form below, enter a  text you want to analyse. Leave it blank and press enter in the input box if you want to use the above default text.

In [6]:
inp_text = input("Please enter a text you want to analyse. Leave it blank if you want to use the default text")
print(inp_text)

Please enter a text you want to analyse. Leave it blank if you want to use the default text



In [7]:
if inp_text != "":
  sentence = inp_text
print(sentence)

  Konni ZILLIACUS: British. A Russian Finn by birth, he came to notice through his Communist associations while working for the League of Nations in 1932. He was elected Labour MP for Gateshead in the 1945 General Election, but was expelled from the party in 1949 for persistently attacking party policy 


Now, we pass the sentence through our model and print extracted named entity.

In [8]:
nlp = pipeline("ner", model=modelNER, tokenizer=tokenizer)

ner_results = nlp(sentence)
named_entities = hf.getNE(ner_results)
print(named_entities)

[('Konni ZIL', 'B-PER'), ('British', 'B-MISC'), ('Russian', 'B-MISC'), ('Finn', 'B-MISC'), ('Communist', 'B-ORG'), ('League of Nations', 'B-ORG'), ('Labour', 'B-LOC'), ('Gateshead', 'B-MISC'), ('General Election', 'B-MISC')]


If you have used the default example, you can see that the model has extracted "Konni ZIL" instead of "Konni ZILLIACUS". This is because the model breaks the words into smaller tokens and it was not able to predict the token "IACUS" is part of a name.

Now we predict the linking for the recognized named entity. It gives a list of Wikipedia page title for the named entity predicted earlier.

In [9]:
outputs = modelNEL.generate(
    **tokenizerNEL(sentence, return_tensors="pt"),
    num_beams=10,
    num_return_sequences=10,
)

tokenizerNEL.batch_decode(outputs, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Conservative Party (UK) >> en',
 'Member of Parliament (United Kingdom) >> en',
 'Gateshead (UK Parliament constituency) >> en',
 'Labour Party (UK) >> en',
 'Politics of the United Kingdom >> en',
 'House of Commons of the United Kingdom >> en',
 'Member of parliament >> en',
 'Parliament of the United Kingdom >> en',
 'United Kingdom >> en',
 'Conservative Party (United Kingdom) >> en']

Now, if we want to get the list for specific named entity then we pass the concerned named entity with [START] and [END] tags. The technical term for this is padding.

In [10]:
import re
sentence_pad = hf.get_pad(sentence, named_entities)
print(sentence_pad)

Konni ZILLIACUS
British
Russian
Finn
Communist
League of Nations
Labour
Gateshead
General Election
  [START]Konni ZILLIACUS[END]: British. A Russian Finn by birth, he came to notice through his Communist associations while working for the League of Nations in 1932. He was elected Labour MP for Gateshead in the 1945 General Election, but was expelled from the party in 1949 for persistently attacking party policy 


In [11]:
text = input(sentence_pad)

  [START]Konni ZILLIACUS[END]: British. A Russian Finn by birth, he came to notice through his Communist associations while working for the League of Nations in 1932. He was elected Labour MP for Gateshead in the 1945 General Election, but was expelled from the party in 1949 for persistently attacking party policy 


In [12]:
print(text)
if text != "":
  sentence_pad = text

In [13]:
outputs = modelNEL.generate(
    **tokenizerNEL(sentence_pad, return_tensors="pt"),
    num_beams=3,
    num_return_sequences=3,
)

tokenizerNEL.batch_decode(outputs, skip_special_tokens=True)

['Konni Zilliacus >> en', 'Konni Zillacus >> en', 'Zilliacus >> en']